In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/emmanuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
f = open('Stopword-List.txt', 'r')
stop = f.read()

In [4]:
import re

pattern = r"[\w]+|[^-_\w\s()@#$%^&*+={[\]};,<>./?~`\"]"

def tokenize(text):
    return re.findall(pattern, text)

In [5]:
from collections import defaultdict
import os
import re
from nltk.stem import PorterStemmer

ps = PorterStemmer()

docs = set()
dic = {}
punc = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '=', '{', '[', ']', '}', ':', ';', "'", '"', ',', '<', '>', '.', '/', '?', '~', '`']

for subdir, dirs, files in os.walk('ResearchPapers'):
    for file in files:
        with open(subdir + os.sep + file, 'r', encoding='cp1252') as txt:
            # Extract document ID from the filename
            doc = re.search('[0-9]*', file).group()
            doc = int(doc)
            docs.add(doc) # Add document ID to the set of docs
            # Read and tokenize the text
            tokens = tokenize(txt.read())
            
            # Process each token
            for t in tokens:
                if t not in stop and t not in punc:
                    # Lowercase and stem the token
                    term = ps.stem(t.lower())

                    # Update the inverted index
                    if term not in dic:
                        # Create a new linked list (assumed to be defined as `LL` class) for the term
                        dic[term] = defaultdict(int)
                    dic[term][doc] += 1

In [6]:
doc_list = list(docs)
doc_map = dict()
for i in range(len(doc_list)):
    doc_map[doc_list[i]] = i
print(doc_map)

{1: 0, 2: 1, 3: 2, 7: 3, 8: 4, 9: 5, 11: 6, 12: 7, 13: 8, 14: 9, 15: 10, 16: 11, 17: 12, 18: 13, 21: 14, 22: 15, 23: 16, 24: 17, 25: 18, 26: 19}


In [7]:
dic_map = dict()
keys = sorted(dic.keys())
for i in range(len(keys)):
    dic_map[keys[i]] = i

In [8]:
import numpy as np
a = np.empty(shape=(len(docs), len(dic)))
a.fill(0)
for key1, value1 in dic.items():
    for key2, value2 in value1.items():
        a[doc_map[key2]][dic_map[key1]] = value2

for n in doc_list:
    print(a[doc_map[n]][dic_map[ps.stem('machine')]])
print('hello')
dic[ps.stem('machine')]

57.0
27.0
12.0
89.0
30.0
0.0
0.0
0.0
5.0
5.0
14.0
22.0
19.0
7.0
3.0
10.0
1.0
24.0
5.0
10.0
hello


defaultdict(int,
            {8: 30,
             15: 14,
             17: 19,
             21: 3,
             25: 5,
             18: 7,
             16: 22,
             3: 12,
             24: 24,
             22: 10,
             7: 89,
             2: 27,
             14: 5,
             23: 1,
             26: 10,
             1: 57,
             13: 5})